In [ ]:
# Prepare Google Colab Environement and build handmade library
!git clone https://github.com/kaenova/Headline_Detection.git
%cd "/content/Headline_Detection"

!make lib-install

In [ ]:
# # Reset Google Colab Environment
# %cd ..
# !rm -fr Headline_Detection

In [2]:
import torch
import math
import torch.nn as nn
import torch.nn.functional as F

from tqdm import tqdm
from transformers import BertForSequenceClassification, BertTokenizerFast
from datasets import load_dataset
from torchmetrics.classification import MulticlassAccuracy
from torch.utils.data import Dataset, DataLoader


# Custom handmade library
import kaelib.processor.preprocessing_func as prep_func
from kaelib.processor import TextProcessingPipeline,NDETCStemmerWraper

# Load Dataset

In [3]:
data = load_dataset("jakartaresearch/google-play-review")
data_train = data.get('train').to_pandas()[['text', 'label']]
data_test = data.get('validation').to_pandas()[['text', 'label']]

Found cached dataset google-play-review (C:/Users/kaeno/.cache/huggingface/datasets/jakartaresearch___google-play-review/default/1.0.0/df84e67f495cc6639ab0bbf74ff0190498a0b22294fdaca26a5b25e090671c29)


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
# Train test split

text_train = data_train["text"].values.tolist()
text_test = data_test["text"].values.tolist()

label2id = {"pos": 1, "neg": 0}
id2label = {v: k for k, v in label2id.items()}

labels_train = data_train["label"].apply(lambda x: label2id[x]).values.tolist()
labels_test = data_test["label"].apply(lambda x: label2id[x]).values.tolist()

# Prep text
pipeline = TextProcessingPipeline([
    prep_func.lowercasing,
    prep_func.remove_html_tags,
    prep_func.remove_url,
    prep_func.remove_punctuation
])

X_train = pipeline.process_corpus(text_train)
X_test = pipeline.process_corpus(text_test)

y_train = labels_train[:]
y_test = labels_test[:]

In [20]:
class IndoBERTDataset(Dataset):
    def __init__(self, texts: 'lst[str]', targets: 'lst[str]'):
        assert len(texts) == len(targets)
        self.tokenizer = BertTokenizerFast.from_pretrained("indobenchmark/indobert-base-p2")
        self.texts = texts
        self.targets = targets

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        bert_tokens = self.tokenizer(self.texts[idx], max_length=256, 
                                     padding='max_length', truncation=True, 
                                     return_tensors='pt' )
        targets = torch.tensor(self.targets[idx], dtype=torch.int64)
        return bert_tokens, targets
    
    @staticmethod
    def squeeze_unwanted_batch(x):
        """
        This function handles an unexpected output of DataLoader that creates extra dimension on dim=1
        """
        new_x = {}
        for keys in x:
            new_x[keys] = x[keys].squeeze()
        return new_x

In [21]:
train_dataset = IndoBERTDataset(X_train, y_train)
test_dataset = IndoBERTDataset(X_test, y_test)

In [27]:
"""
https://arxiv.org/pdf/1810.04805.pdf

We use a batch size of 32 and fine-tune for 3
epochs over the data for all GLUE tasks. For each
task, we selected the best fine-tuning learning rate
(among 5e-5, 4e-5, 3e-5, and 2e-5)
"""

batch_size = 32
epochs = 3
device = "cpu"
lr = 5e-5 

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

num_mini_batch = math.ceil(len(X_train) / batch_size)

In [31]:
model = BertForSequenceClassification.from_pretrained(
    "indobenchmark/indobert-base-p2",
    num_labels=2,
    id2label=id2label,
    label2id=label2id,
    problem_type="multi_label_classification",
).to(device)

optim = torch.optim.Adam(model.parameters(), lr=lr)

metrics = MulticlassAccuracy(2).to(device)

# Half precision
scaler = torch.cuda.amp.GradScaler()

for i in range(epochs):
    # Progress bar
    pbar = tqdm(total=num_mini_batch)
    pbar.set_description(f"EPOCH {i + 1} / {epochs}: Training")
    
    epoch_loss = []
    avg_loss = 0
    epoch_loss_test = []
    avg_loss_test = 0
    epoch_metrics_test = []
    avg_metric_test = 0

    for train_tokens, train_target in train_loader:
        model.zero_grad()

        # Forward
        train_tokens = IndoBERTDataset.squeeze_unwanted_batch(train_tokens)
        with torch.cuda.amp.autocast():
            pred = model(**train_tokens).logits
            loss = F.cross_entropy(pred, train_target)

        # Backprop
        scaler.scale(loss).backward()
        scaler.step(optim)
        scaler.update()

        # Metrics and logging
        epoch_loss.append(loss.item())
        avg_loss = sum(epoch_loss) / len(epoch_loss)
        pbar.set_postfix(
            {
                "loss": f"{avg_loss:.4f}",
                "loss_test": f"{avg_loss_test:.4f}",
                "metric_test": f"{avg_metric_test:.4f}",
            }
        )
        pbar.update(1)

    for test_tokens, test_target in test_loader:
        test_tokens = IndoBERTDataset.squeeze_unwanted_batch(test_tokens)

        # Forward test
        with torch.no_grad():
            pred_test = model(**test_tokens).logits
            loss_test = F.cross_entropy(pred_test, test_target)
            metrics_test = metrics(pred_test, test_target)

        epoch_loss_test.append(loss_test.item())
        avg_loss_test = sum(epoch_loss_test) / len(epoch_loss_test)
        epoch_metrics_test.append(metrics_test.item())
        avg_metric_test = sum(epoch_metrics_test) / len(epoch_metrics_test)

        pbar.set_description(f"EPOCH {i + 1} / {epochs}: Infering on Testing Data")
        pbar.set_postfix(
            {
                "loss": f"{avg_loss:.4f}",
                "loss_test": f"{avg_loss_test:.4f}",
                "metric_test": f"{avg_metric_test:.4f}",
            }
        )
    
    pbar.close()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
d:\@Programming\@On Going\Data\Headline_Detection\venv\lib\site-packages\torch\cuda\amp\grad_scaler.py:118: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
EPOCH 1 / 3: Training:   0%|          | 0/220 [00:00<?, ?it/s]d:\@Programming\@On Going\Data\Headline_Detection\venv\lib\site-packages\torch\amp\autocast_mode.py:202: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


KeyboardInterrupt: 